In [1]:
from pathlib import Path
import pandas as pd
import os
import numpy as np
import time
import datetime
import h5py

%load_ext autoreload
%autoreload 2

In [2]:
root_dir = Path.cwd().parent.parent
print(root_dir)
path_data_raw_folder = Path(root_dir / 'data/raw/airbus/' )
print(path_data_raw_folder)

/home/tim/Documents/PyPHM
/home/tim/Documents/PyPHM/data/raw/airbus


**Training Data Description**

The training dataset is composed of 1677 one-minute-sequences @1024Hz of accelerometer data measured on test helicopters at various locations, in various angles (X, Y, Z), on different flights. All data has been multiplied by a factor so that absolute values are meaningless, but no other normalization procedure was carried out. All sequences are considered as normal and should be used to learn normal behaviour of accelerometer data.

In [5]:
# load dftrain.h5 with pandas
df_train = pd.read_hdf(path_data_raw_folder / 'dftrain.h5', 'dftrain')

# add y column of all zeros (indicating no anomaly)
df_train['y'] = 0

print(df_train.shape)
df_train.head()


(1677, 61441)


,0,1,2,3,4,5,6,7,8,9,...,61431,61432,61433,61434,61435,61436,61437,61438,61439,y
0,0.041259,0.041259,0.032573,0.023887,0.029315,0.041259,0.045602,0.038001,0.030401,0.032573,...,-0.007600,-0.003257,0.065145,0.047773,-0.009772,0.031487,0.096632,0.077089,0.040173,0
1,-0.211722,-0.264924,-0.274696,-0.236694,-0.156349,-0.059716,0.005429,0.046687,0.153091,0.281210,...,-0.004343,-0.241037,-0.636252,-0.953292,-0.980436,-0.846888,-0.838202,-0.880546,-0.739398,0
2,0.214105,0.154930,0.136640,0.013987,-0.038733,-0.015063,-0.111894,-0.104363,0.047340,-0.054871,...,0.375490,0.699337,0.965085,1.086662,1.132926,1.279249,1.296464,0.937112,0.451879,0
3,-0.154837,-0.127768,-0.217638,-0.284770,-0.299929,-0.270694,-0.077960,0.092036,0.076877,0.154837,...,-0.171079,-0.036814,-0.024904,0.031400,0.140761,-0.011911,-0.173244,-0.063884,0.081208,0
4,-1.022780,-0.916376,-0.676425,-0.461445,-0.330069,-0.122690,0.178064,0.489675,0.799115,0.931577,...,1.134613,0.836030,0.479903,0.109661,-0.285553,-0.628651,-0.916376,-1.010837,-0.804544,0


In [4]:
# the frequency of data sampling is 1024 Hz
freq = 1024

# each signal is 1 minute in length
meas_time = 60

print(freq * meas_time)
df_train.shape

61440


(1677, 61441)

In [5]:
# load dfvalid.h5 with pandas
df_val = pd.read_hdf(path_data_raw_folder / 'dfvalid.h5', 'dfvalid')
print(df_val.shape)
df_val.head()

(594, 61440)


,0,1,2,3,4,5,6,7,8,9,...,61430,61431,61432,61433,61434,61435,61436,61437,61438,61439
0,0.070650,0.085524,0.076847,0.085524,0.120229,0.140061,0.107834,0.048340,0.014874,0.030987,...,0.018592,-0.017353,-0.049579,-0.061974,-0.022311,0.052058,0.091721,0.075608,0.045861,0.027268
1,-0.300685,-0.882656,-1.733789,-2.181179,-1.942328,-1.719240,-1.981126,-2.323034,-2.325459,-2.349708,...,0.454665,-0.782024,-1.725302,-1.829572,-1.170004,-0.278861,-0.152767,-1.035424,-2.312123,-3.199628
2,-0.056398,-0.152028,-0.058850,0.140994,0.170419,0.154480,0.270954,0.359228,0.334708,0.349420,...,0.023295,-0.019617,-0.062528,-0.094405,-0.105439,-0.123830,-0.166741,-0.159385,-0.106665,-0.131186
3,0.030618,0.020412,0.058747,0.124712,0.155828,0.161553,0.167777,0.170515,0.165038,0.141141,...,-0.073682,-0.052524,-0.023897,-0.009210,0.016429,0.058000,0.097828,0.114755,0.090360,0.054764
4,-1.071597,-6.912540,-13.677612,-16.463519,-15.790084,-11.653620,-7.668311,-9.942997,-14.321553,-13.704648,...,-5.227723,-6.107613,-6.577051,-6.123589,-5.324806,-4.965969,-4.464579,-3.442138,-3.794831,-6.133420


In [6]:
# load the dfvalid_groundtruth.csv as dataframe
df_val_labels = pd.read_csv(path_data_raw_folder / 'dfvalid_groundtruth.csv', dtype={'seqID': int, 'anomaly': int}, )
df_val_labels.head()

,seqID,anomaly
0,0,0
1,1,1
2,2,0
3,3,0
4,4,1


In [7]:
# append the anomaly label to the df_val dataframe
df_val = df_val.merge(df_val_labels, left_index=True, right_on='seqID')

# drop the seqID column and rename the anomaly column to y
df_val = df_val.drop('seqID', axis=1).rename(columns={'anomaly': 'y'})

In [8]:
df_val.head()

,0,1,2,3,4,5,6,7,8,9,...,61431,61432,61433,61434,61435,61436,61437,61438,61439,y
0,0.070650,0.085524,0.076847,0.085524,0.120229,0.140061,0.107834,0.048340,0.014874,0.030987,...,-0.017353,-0.049579,-0.061974,-0.022311,0.052058,0.091721,0.075608,0.045861,0.027268,0
1,-0.300685,-0.882656,-1.733789,-2.181179,-1.942328,-1.719240,-1.981126,-2.323034,-2.325459,-2.349708,...,-0.782024,-1.725302,-1.829572,-1.170004,-0.278861,-0.152767,-1.035424,-2.312123,-3.199628,1
2,-0.056398,-0.152028,-0.058850,0.140994,0.170419,0.154480,0.270954,0.359228,0.334708,0.349420,...,-0.019617,-0.062528,-0.094405,-0.105439,-0.123830,-0.166741,-0.159385,-0.106665,-0.131186,0
3,0.030618,0.020412,0.058747,0.124712,0.155828,0.161553,0.167777,0.170515,0.165038,0.141141,...,-0.052524,-0.023897,-0.009210,0.016429,0.058000,0.097828,0.114755,0.090360,0.054764,0
4,-1.071597,-6.912540,-13.677612,-16.463519,-15.790084,-11.653620,-7.668311,-9.942997,-14.321553,-13.704648,...,-6.107613,-6.577051,-6.123589,-5.324806,-4.965969,-4.464579,-3.442138,-3.794831,-6.133420,1


In [9]:
x = df_val.drop('y', axis=1).to_numpy()
y = df_val['y'].to_numpy()
x.shape

(594, 61440)

In [10]:
max_range = 10
a = list(range(0,max_range))
b = [0] * max_range

[str(i) + "_" + str (j) for i, j in list(zip(a,b))]

['0_0', '1_0', '2_0', '3_0', '4_0', '5_0', '6_0', '7_0', '8_0', '9_0']

In [23]:
window_size = 512
stride = 256

# instantiate the "temporary" lists to store the windows and labels
window_list = []
window_id_list = []
window_label_list = []

# fit the strided windows into the dummy_array until the length
# of the window does not equal the proper length (better way to do this???)

n_signals = x.shape[0]
len_signal = x.shape[1]


for i in range(len_signal):
    windowed_signal = x[:,
        i * stride : i * stride + window_size
    ]


    # if the windowed signal is the proper length, add it to the list
    if windowed_signal.shape == (n_signals, window_size):
        window_list.append(windowed_signal)

        window_id_list.append(np.array([str(j) + "_" + str (k) for j, k in list(zip(list(range(0,n_signals)), [i] * n_signals))]))

        window_label_list.append(y)

        # # create sub_cut_id fstring to keep track of the cut_id and the window_id
        # window_id_list.append(f"{cut_no}_{i}")

        # # create the sub_cut_label and append it to the list
        # window_label_list.append(int(label))

    else:
        break

# window_array = np.array(window_list).reshape(-1, window_size, 1)

x = np.vstack(window_list,)
window_id_array = np.hstack(window_id_list)

# window_id_array = np.array(window_id_list).reshape(n_signals, len(window_id_list))
window_label_array = np.hstack(window_label_list)
y_array = np.vstack((window_label_list, window_id_list)).reshape(-1, 2)
# window_id_array = np.stack(window_id_list, axis=0).reshape(n_signals, len(window_id_list), 1)

In [24]:
print(window_array.shape)
print(window_id_array.shape)
print(window_label_array.shape)
print(y_array.shape)

(141966, 512)
(141966,)
(141966,)
(141966, 2)


In [55]:
window_array[1,:10]

array([-0.30068505, -0.88265611, -1.73378878, -2.18117904, -1.94232841,
       -1.71923951, -1.98112648, -2.32303448, -2.32545936, -2.34970816])

In [42]:
df_val.head()

,0,1,2,3,4,5,6,7,8,9,...,61431,61432,61433,61434,61435,61436,61437,61438,61439,y
0,0.070650,0.085524,0.076847,0.085524,0.120229,0.140061,0.107834,0.048340,0.014874,0.030987,...,-0.017353,-0.049579,-0.061974,-0.022311,0.052058,0.091721,0.075608,0.045861,0.027268,0
1,-0.300685,-0.882656,-1.733789,-2.181179,-1.942328,-1.719240,-1.981126,-2.323034,-2.325459,-2.349708,...,-0.782024,-1.725302,-1.829572,-1.170004,-0.278861,-0.152767,-1.035424,-2.312123,-3.199628,1
2,-0.056398,-0.152028,-0.058850,0.140994,0.170419,0.154480,0.270954,0.359228,0.334708,0.349420,...,-0.019617,-0.062528,-0.094405,-0.105439,-0.123830,-0.166741,-0.159385,-0.106665,-0.131186,0
3,0.030618,0.020412,0.058747,0.124712,0.155828,0.161553,0.167777,0.170515,0.165038,0.141141,...,-0.052524,-0.023897,-0.009210,0.016429,0.058000,0.097828,0.114755,0.090360,0.054764,0
4,-1.071597,-6.912540,-13.677612,-16.463519,-15.790084,-11.653620,-7.668311,-9.942997,-14.321553,-13.704648,...,-6.107613,-6.577051,-6.123589,-5.324806,-4.965969,-4.464579,-3.442138,-3.794831,-6.133420,1


In [25]:
window_label_array.shape

(141966,)

In [23]:
window_array[0]

array([[ 0.07065007],
       [ 0.08552377],
       [ 0.07684745],
       [ 0.08552377],
       [ 0.12022907],
       [ 0.14006067],
       [ 0.10783432],
       [ 0.04833952],
       [ 0.0148737 ],
       [ 0.03098687],
       [ 0.05825532],
       [ 0.05205795],
       [ 0.0297474 ],
       [ 0.03842372],
       [ 0.07188955],
       [ 0.07065007],
       [ 0.02355002],
       [ 0.00123947],
       [ 0.0396632 ],
       [ 0.0842843 ],
       [ 0.07684745],
       [ 0.0446211 ],
       [ 0.02850792],
       [ 0.00867632],
       [-0.04338162],
       [-0.08676325],
       [-0.07312902],
       [-0.0148737 ],
       [ 0.03222635],
       [ 0.05329742],
       [ 0.07560797],
       [ 0.099158  ],
       [ 0.09296062],
       [ 0.04214215],
       [-0.02726845],
       [-0.07065007],
       [-0.06817112],
       [-0.04338162],
       [-0.02231055],
       [-0.00371842],
       [ 0.02602897],
       [ 0.05825532],
       [ 0.0545369 ],
       [ 0.01239475],
       [-0.0198316 ],
       [-0

In [15]:
np.array(window_id_list).shape

(239, 594)

In [23]:
len(window_id_list)

239

In [21]:
window_list[0].shape


(594, 512)